## Initial

In [29]:
import os
from dotenv import load_dotenv

# Load .env file
load_dotenv()

HF_TOKEN = os.getenv("HF_TOKEN")
OPENROUTER_API_KEY = os.getenv("SHISHA_API_KEY")
GROQCLOUD_API_KEY = os.getenv("GROQCLOUD_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [30]:
import requests
from openai import OpenAI 

OPENROUTER_CLIENT = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=OPENROUTER_API_KEY
)

OPENAI_CLIENT = OpenAI(api_key=OPENAI_API_KEY)

def call_openai_api(prompt: str) -> str:
    try:
        response = OPENAI_CLIENT.chat.completions.create(
            model="gpt-4.1-nano",
            messages=[
                {
                    "role": "user", 
                    "content": prompt
                }
            ],
        )

        reply = response.choices[0].message.content

        return reply 
    
    except Exception as e:
        print(f"API called failed: {e}")
        return None

def call_openrouter_api(prompt: str) -> str:
    try:
        completion = OPENROUTER_CLIENT.chat.completions.create(
            model="moonshotai/kimi-dev-72b:free",
            messages=[
                {
                    "role": "user",
                    "content": prompt
                }
            ]
        )

        return completion.choices[0].message.content
    
    except Exception as e:
        print(f"API called failed: {e}")
        return None
    
def call_groq_api(prompt: str) -> str:
    url = "https://api.groq.com/openai/v1/chat/completions"
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {GROQCLOUD_API_KEY}"
    }
    payload = {
        "model": "meta-llama/llama-4-maverick-17b-128e-instruct",
        "messages": [
            {
                "role": "user",
                "content": prompt
            }
        ]
    }

    try:
        response = requests.post(url, headers=headers, json=payload)
        response.raise_for_status()
        result = response.json()
        
        return result["choices"][0]["message"]["content"]
    except Exception as e:
        print(f"GROQ API call failed: {e}")
        return None

In [31]:
MULTI_TURN_PROMPT = """
Given one base question and answer about a specific hotel:

Question: {question} 
Answer: {answer}

Your task:
- Generate 2 short multi-turn conversations between a user and the assistant.
- Each conversation should contain 2–4 exchanges (turns), starting from the base question.
- The follow-up questions must stay on-topic and explore related details (clarifications, follow-ups, polite confirmations, etc.).
- Do NOT add new facts that are not mentioned in the original answer.
- Keep the language natural and realistic, like how a real user might ask.
- Return the result in valid JSON format, where each conversation is a separate object:

json
[
    {{
        "conversations": [
            {{"from": "human", "value": "..." }},
            {{"from": "gpt", "value": "..." }},
            {{"from": "human", "value": "..." }},
            {{"from": "gpt", "value": "..." }},
            ...
        ]
    }},
    ...
]
"""

## Top-3 Question-Answer

In [32]:
import json

file_path = "./0-preprocessing.json"
list_questions = []
list_answers = []
list_hotels_name = []

with open(file_path, 'r', encoding='utf-8') as file:
    raw_data = json.load(file)

for data in raw_data:
    list_questions.append(data['conversations'][0]['value'])
    list_answers.append(data['conversations'][1]['value'])
    list_hotels_name.append(data['hotel_name'])

for question, answer, hotel_name in zip(list_questions[:3], list_answers[:3], list_hotels_name[:3]):
    print(hotel_name)
    print(question)
    print(answer)
    print("\n===\n")

Renaissance Riverside Saigon Hotel
Bạn có thể cung cấp thông tin tổng quan về Renaissance Riverside Saigon Hotel được không?
Renaissance Riverside Saigon Hotel là khách sạn 5 sao tọa lạc ngay trung tâm Thành phố Hồ Chí Minh. Khách sạn có 336 phòng với cửa sổ nhìn ra thành phố, nhiều tiện nghi hiện đại cùng 2 nhà hàng và bar sang trọng. Vị trí thuận tiện cách sân bay Tân Sơn Nhất khoảng 30 phút lái xe và gần các điểm tham quan nổi tiếng như Nhà hát Lớn, Chợ Bến Thành.

===

Renaissance Riverside Saigon Hotel
Khách sạn Renaissance Riverside Saigon nằm ở địa chỉ nào vậy?
Renaissance Riverside Saigon Hotel tọa lạc tại địa chỉ 8 - 15, Ton Duc Thang, Quận 1, Thành phố Hồ Chí Minh, Việt Nam.

===

Renaissance Riverside Saigon Hotel
Bạn có thể cho tôi biết các hạng phòng hiện có tại Renaissance Riverside Saigon Hotel không?
Khách sạn Renaissance Riverside Saigon có nhiều hạng phòng đa dạng như: Standard Room, Suite, Guest Room, Club Guest Room, Executive Studio, Executive Suite, R Club Room, D

## Send each question to LLM for multi-turn conversations

In [33]:
from typing import Dict, List
import json

def extract_list_conversations(response: str, hotel_name: str) -> List[Dict]:
    print(response)
    list_conversations = json.loads(response)

    for conv in list_conversations:
        conv['hotel_name'] = hotel_name

    # list_conversations = [q.lstrip('-').strip() for q in list_conversations if q.strip()]

    return list_conversations

In [34]:
list_conversations = []

for question, answer, hotel_name in zip(list_questions, list_answers, list_hotels_name):
    prompt = MULTI_TURN_PROMPT.format(question=question, answer=answer)
    response = call_openai_api(prompt)

    try:
        new_conversations = extract_list_conversations(response, hotel_name)
    except Exception as e:
        print(f"Error extracting conversations: {e}")
        continue

    list_conversations.extend(new_conversations)

(list_conversations)

[
    {
        "conversations": [
            {
                "from": "human",
                "value": "Bạn có thể cung cấp thông tin tổng quan về Renaissance Riverside Saigon Hotel được không?"
            },
            {
                "from": "gpt",
                "value": "Renaissance Riverside Saigon Hotel là khách sạn 5 sao tọa lạc ngay trung tâm Thành phố Hồ Chí Minh. Khách sạn có 336 phòng với cửa sổ nhìn ra thành phố, nhiều tiện nghi hiện đại cùng 2 nhà hàng và bar sang trọng. Vị trí thuận tiện cách sân bay Tân Sơn Nhất khoảng 30 phút lái xe và gần các điểm tham quan nổi tiếng như Nhà hát Lớn, Chợ Bến Thành."
            },
            {
                "from": "human",
                "value": "Các phòng đều có view ra thành phố đúng không? Có loại phòng nào đặc biệt không?"
            },
            {
                "from": "gpt",
                "value": "Đúng vậy, tất cả các phòng đều có cửa sổ nhìn ra thành phố. Tuy nhiên, trong thông tin hiện tại không đề cập đế

[{'conversations': [{'from': 'human',
    'value': 'Bạn có thể cung cấp thông tin tổng quan về Renaissance Riverside Saigon Hotel được không?'},
   {'from': 'gpt',
    'value': 'Renaissance Riverside Saigon Hotel là khách sạn 5 sao tọa lạc ngay trung tâm Thành phố Hồ Chí Minh. Khách sạn có 336 phòng với cửa sổ nhìn ra thành phố, nhiều tiện nghi hiện đại cùng 2 nhà hàng và bar sang trọng. Vị trí thuận tiện cách sân bay Tân Sơn Nhất khoảng 30 phút lái xe và gần các điểm tham quan nổi tiếng như Nhà hát Lớn, Chợ Bến Thành.'},
   {'from': 'human',
    'value': 'Các phòng đều có view ra thành phố đúng không? Có loại phòng nào đặc biệt không?'},
   {'from': 'gpt',
    'value': 'Đúng vậy, tất cả các phòng đều có cửa sổ nhìn ra thành phố. Tuy nhiên, trong thông tin hiện tại không đề cập đến các loại phòng đặc biệt hay suite, chỉ biết rằng có nhiều tiện nghi hiện đại trong các phòng tiêu chuẩn.'}],
  'hotel_name': 'Renaissance Riverside Saigon Hotel'},
 {'conversations': [{'from': 'human',
    '

In [35]:
list_conversations[:2]  # Display the first two conversations to verify

[{'conversations': [{'from': 'human',
    'value': 'Bạn có thể cung cấp thông tin tổng quan về Renaissance Riverside Saigon Hotel được không?'},
   {'from': 'gpt',
    'value': 'Renaissance Riverside Saigon Hotel là khách sạn 5 sao tọa lạc ngay trung tâm Thành phố Hồ Chí Minh. Khách sạn có 336 phòng với cửa sổ nhìn ra thành phố, nhiều tiện nghi hiện đại cùng 2 nhà hàng và bar sang trọng. Vị trí thuận tiện cách sân bay Tân Sơn Nhất khoảng 30 phút lái xe và gần các điểm tham quan nổi tiếng như Nhà hát Lớn, Chợ Bến Thành.'},
   {'from': 'human',
    'value': 'Các phòng đều có view ra thành phố đúng không? Có loại phòng nào đặc biệt không?'},
   {'from': 'gpt',
    'value': 'Đúng vậy, tất cả các phòng đều có cửa sổ nhìn ra thành phố. Tuy nhiên, trong thông tin hiện tại không đề cập đến các loại phòng đặc biệt hay suite, chỉ biết rằng có nhiều tiện nghi hiện đại trong các phòng tiêu chuẩn.'}],
  'hotel_name': 'Renaissance Riverside Saigon Hotel'},
 {'conversations': [{'from': 'human',
    '

## Save to file

In [36]:
from typing import Dict


def save_to_file(data: List[Dict], filename: str):
    with open(filename, 'w', encoding='utf-8') as file:
        file.write("[")

        for index, entry in enumerate(data):
            char = ','
            if index == len(data) - 1:
                char = ''

            file.write(json.dumps(entry, ensure_ascii=False) + f"{char}\n")

        file.write("]")
        
    print(f'Saved {len(data)} entries to {filename}')

In [37]:
save_to_file(list_conversations, '2-multi_turn_conversations.json')

Saved 578 entries to 2-multi_turn_conversations.json
